# VGG16 implementation using keras on DogsVsCats datset

In [1]:
import numpy as np
from keras import layers
from keras.models import Model
import matplotlib.pyplot as plt
import cv2
import pickle as pk
import os
from tqdm import tqdm
from random import shuffle


%matplotlib inline

Using TensorFlow backend.


Download dataset from [http://files.fast.ai/data/dogscats.zip](http://files.fast.ai/data/dogscats.zip)

In [2]:
train_path = "./dogscats/train/"
validation_path = "./dogscats/valid/"

In [3]:
training_data = []
validation_data = []

def insert_data(data_type, dir_path, category):
    for img in tqdm(os.listdir(dir_path+category)):
        path = os.path.join(dir_path+category,img)
        image = cv2.imread(path)
        image = cv2.resize(image, (224, 224))
        label = [1,0] if category=="cats" else [0,1]
        data_type.append([np.array(image), np.array(label)])
        

insert_data(training_data, train_path, "cats")
insert_data(training_data, train_path, "dogs")
insert_data(validation_data, validation_path, "cats")
insert_data(validation_data, validation_path, "dogs")

shuffle(training_data)
shuffle(validation_data)


train_x = []
train_y = []
val_x = []
val_y = []

for i in range(len(training_data)/20):
    train_x.append(training_data[i][0])
    train_y.append(training_data[i][1])
    

for i in range(len(validation_data)/20):
    val_x.append(validation_data[i][0])
    val_y.append(validation_data[i][1])
    
train_x = np.array(train_x)
train_y = np.array(train_y)
val_x = np.array(val_x)
val_y = np.array(val_y)

n_train = train_x.shape[0]
n_val = val_x.shape[0]

print train_x.shape 
print train_y.shape 
print val_x.shape
print val_y.shape

100%|██████████| 1000/1000 [00:01<00:00, 543.43it/s]


(1150, 224, 224, 3)
(1150, 2)
(100, 224, 224, 3)
(100, 2)


In [4]:
def VGG16(input_shape):
    
    X_input = layers.Input(input_shape)
    
    X = layers.Conv2D(64, (3,3), strides=(1,1), padding = "same", name="conv1")(X_input)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(64, (3,3), strides=(1,1), padding = "same", name="conv2")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(X)
    
    
    X = layers.Conv2D(128, (3,3), strides=(1,1), padding = "same", name="conv3")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(128, (3,3), strides=(1,1), padding = "same", name="conv4")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(X)
    
    
    X = layers.Conv2D(256, (3,3), strides=(1,1), padding = "same", name="conv5")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(256, (3,3), strides=(1,1), padding = "same", name="conv6")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(256, (3,3), strides=(1,1), padding = "same", name="conv7")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(X)
    
    
    X = layers.Conv2D(512, (3,3), strides=(1,1), padding = "same", name="conv8")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(512, (3,3), strides=(1,1), padding = "same", name="conv9")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(512, (3,3), strides=(1,1), padding = "same", name="conv10")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(X)
    
    
    X = layers.Conv2D(512, (3,3), strides=(1,1), padding = "same", name="conv11")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(512, (3,3), strides=(1,1), padding = "same", name="conv12")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(512, (3,3), strides=(1,1), padding = "same", name="conv13")(X)
    X = layers.Activation('relu')(X)
    
    X = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(X)
    
    X = layers.Flatten()(X)
    
    X = layers.Dense(4096, activation='relu', name="fc14")(X)
    
    X = layers.Dense(4096, activation='relu', name="fc15")(X)
    
    X = layers.Dense(2, activation='softmax', name="fc16")(X)
    
    model = Model(inputs = X_input, outputs = X, name="VGG16")
    
    return model


In [5]:
vgg16_model = VGG16((224,224,3))

In [6]:
vgg16_model.compile(optimizer = "adam", loss="mean_squared_error", metrics=["accuracy"])

In [7]:
vgg16_model.fit(x = train_x, y = train_y, epochs = 1, batch_size=32)

Epoch 1/1
1150/1150 [==============================] - 2922s 3s/step - loss: 0.4652 - acc: 0.5304
